In [265]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots

In [267]:
class optionPricer:
    # allowed stuff
    types = {'put', 'call'}
    styles = {'european'}
    methods = {'euler', 'milstein'}
    varReductionMethods = {'none', 'antithetic', 'importance', 'control'}
    
    def __init__(self, optionStyle, optionType, initialPrice, strike, std, r = 0.07, T = 1):
        #checking to see if it calcualtable 
        if optionType not in self.types:
            raise ValueError('type not allowed')
        if optionStyle not in self.styles:
            raise ValueError('style not allowed')

        # setting attributes
        self.optionType = optionType.lower()
        self.optionStyle = optionStyle.lower()
        self.initialPrice = initialPrice
        self.strike = strike
        self.std = std
        self.r = r
        self.T = T

    def getPaths(self, method = 'euler', plot = False, varReduction = 'None', npaths = 1000, nt = 1000):
        '''
        gets all the stock paths that may be needed
        can specify method and if you want it plotted
        '''
        if method.lower() not in self.methods:
            raise ValueError('method not allowed')
        if varReduction.lower() not in self.varReductionMethods:
            raise ValueError('variance reduction method not allowed')
        
        dt = self.T/nt
        paths = []

        # antithetical variable variance reduction
        if varReduction == 'antithetic':
            pass

        # no variance reduction applied
        else:
            for i in range(npaths):
                S = np.zeros(nt + 1)
                S[0] = self.initialPrice

                for t in range(nt):
                    change = np.random.normal(loc = 0, scale = 1)
                    if method.lower() == 'euler':
                        S[t+1] = S[t] + self.r * S[t] * dt + self.std * S[t] * np.sqrt(dt) * change
                    if method.lower() == 'milstein':
                        # i think the milstein formula in lec notes is incorrect...
                        S[t+1] = S[t] + self.r * S[t] * dt + self.std * S[t] * np.sqrt(dt) * change + 0.5 * self.std**2 * dt * (change**2 - 1) * S[t]

                paths.append(S)

        if plot:
            fig, ax = subplots(figsize = (16, 8))
            for i in range(len(paths)):
                ax.plot(paths[i])

            ax.set_title(f'Paths generated using {method.capitalize()} Method')
            ax.set_ylabel('Price')
            ax.set_xlabel('Time')
            ax.set_xlim(0, nt)
            ax.axhline(self.initialPrice, color = 'k', ls = '--')

        self.paths = np.array(paths)

    def getPrice(self, varReduction = None, npaths = 1000, nt = 1000):
        '''
        prices the option --> GET PATHS FIRST then put it through this method
        can enter a variance reduction technique
        '''
        # pricing european options
        if self.optionStyle == 'european':
            S_T = self.paths[:, -1] 
            if self.optionType == 'put':
                payoffs = np.maximum(0, self.strike - S_T)
            if self.optionType == 'call':
                payoffs = np.maximum(0, S_T - self.strike)    
            price = np.e ** (-self.r * self.T) * np.mean(payoffs)

        return price
        
        

In [269]:
europut = optionPricer('european', 'put', 100, 100, 0.25)
europut.getPaths(method = 'euler');
europut.getPrice()

6.807667358247195